## 测试: Fugashi 智能读音匹配

测试使用 fugashi 来自动选择最合适的读音

In [7]:
# 重新导入模块以加载新功能
import importlib
import mdx_utils.audio_lookup
importlib.reload(mdx_utils.audio_lookup)

from mdx_utils.audio_lookup import get_word_reading_with_fugashi, match_best_pitch, extract_pitch_info_nhk_old
import re

# 测试单词
test_words = ["精霊", "政権", "庇う"]

for word in test_words:
    print(f"🔍 单词: {word}")
    
    # 获取 fugashi 读音
    fugashi_reading = get_word_reading_with_fugashi(word)
    print(f"   Fugashi 读音: {fugashi_reading}")
    
    # 获取所有候选读音
    if globals().get('nhk_old_mdx') and globals()['nhk_old_mdx'].exists():
        all_pitches = extract_pitch_info_nhk_old(nhk_old_mdx, word, return_all=True)
        
        if all_pitches:
            print(f"   所有候选读音 ({len(all_pitches)} 个):")
            for i, (reading, pitch) in enumerate(all_pitches, 1):
                plain = re.sub(r'<[^>]+>', '', reading)
                print(f"      {i}. {plain} {pitch}")
            
            # 智能匹配
            best_reading, best_pitch = extract_pitch_info_nhk_old(nhk_old_mdx, word, return_all=False)
            if best_reading:
                plain_best = re.sub(r'<[^>]+>', '', best_reading)
                print(f"   ✅ 智能匹配结果: {plain_best} {best_pitch}")
                
                # 显示渲染效果
                display(HTML(f'<div style="font-size: 20px; padding: 10px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 5px;">{best_reading} <span style="color: #4caf50; font-weight: bold;">{best_pitch}</span></div>'))
    
    print()

🔍 单词: 精霊
   Fugashi 读音: セイレイ

🔍 单词: 政権
   Fugashi 读音: セイケン

🔍 单词: 庇う
   Fugashi 读音: カバウ



In [8]:
# 测试 fugashi 是否可用
try:
    import fugashi
    tagger = fugashi.Tagger()
    result = tagger("精霊")
    print("Fugashi 测试:")
    for word in result:
        print(f"  表層形: {word.surface}")
        print(f"  Feature: {word.feature}")
        
        # 尝试不同的读音字段
        try:
            kana = word.feature.kana
            print(f"  読み (kana): {kana}")
        except:
            print(f"  読み (kana): 不可用")
        
        try:
            reading = word.feature[7] if len(word.feature) > 7 else None
            print(f"  読み (feature[7]): {reading}")
        except:
            print(f"  読み (feature[7]): 不可用")
        
        print()
except Exception as e:
    print(f"Fugashi 错误: {e}")

Fugashi 测试:
  表層形: 精霊
  Feature: UnidicFeatures29(pos1='名詞', pos2='普通名詞', pos3='一般', pos4='*', cType='*', cForm='*', lForm='セイレイ', lemma='精霊', orth='精霊', pron='セーレー', orthBase='精霊', pronBase='セーレー', goshu='漢', iType='*', iForm='*', fType='*', fForm='*', iConType='*', fConType='*', type='体', kana='セイレイ', kanaBase='セイレイ', form='セイレイ', formBase='セイレイ', aType='0', aConType='C2', aModType='*', lid='5507737245065728', lemma_id='20037')
  読み (kana): セイレイ
  読み (feature[7]): 精霊



# MDX 音频提取测试 - 使用 mdxscraper.core.audio

本 Notebook 演示如何使用 `mdx_utils` 提取音频和音调信息。

**技术栈:**
- `mdxscraper.core.audio` - 底层音频提取
- `mdx_utils.audio_lookup` - 高层封装API

**词典优先级:**
1. 新版 NHK (AAC 音频)
2. 旧版 NHK (MP3 + 完整音调标记)
3. 大辞泉 第二版 (音频 + 下降位置)

---

In [9]:
# 导入模块
from pathlib import Path
from IPython.display import HTML, display

# 检查 mdxscraper.core.audio 是否可用
try:
    from mdxscraper.core.audio import (
        AudioInfo,
        get_audio_info,
        extract_audio_paths_from_html,
    )
    print("✅ mdxscraper.core.audio 可用")
    AUDIO_MODULE_AVAILABLE = True
except ImportError as e:
    print(f"⚠️  mdxscraper.core.audio 不可用: {e}")
    print("   请确保 mdxscraper 版本支持 audio 模块")
    AUDIO_MODULE_AVAILABLE = False

# 导入 mdx_utils
from mdx_utils import (
    AudioLookup,
    extract_audio_from_mdx,
    extract_pitch_info_nhk_old,
    get_all_audio_info_from_mdx,
)

print("✅ mdx_utils 音频模块已导入")

✅ mdxscraper.core.audio 可用
✅ mdx_utils 音频模块已导入


## Test 1: 底层 API - 单词典音频提取

使用 `extract_audio_from_mdx()` 从单个词典提取音频

In [10]:
# 配置词典路径
DICTS_DIR = Path(r"E:\repos\Japanese-Anime-Text-Analysis\Anime_Texts\JP_Dics")

# 测试词典
test_dicts = [
    (DICTS_DIR / "DJS_N" / "DJS.mdx", "大辞泉 第二版"),
    # 添加其他词典...
]

# 测试单词
word = "政権"

print(f"🔍 测试单词: {word}\n")
print("=" * 60)

for mdx_path, dict_name in test_dicts:
    if not mdx_path.exists():
        print(f"❌ {dict_name}: 文件不存在")
        continue
    
    print(f"\n📚 测试词典: {dict_name}")
    print(f"   路径: {mdx_path.name}")
    
    # 提取音频
    audio_base64, mime_type, source = extract_audio_from_mdx(mdx_path, word, dict_name)
    
    if audio_base64:
        print(f"   ✅ 找到音频!")
        print(f"   - MIME类型: {mime_type}")
        print(f"   - Base64长度: {len(audio_base64)} 字符")
        print(f"   - 来源: {source}")
    else:
        print(f"   ℹ️  未找到音频")

🔍 测试单词: 政権


📚 测试词典: 大辞泉 第二版
   路径: DJS.mdx
   ✅ 找到音频!
   - MIME类型: audio/aac
   - Base64长度: 8708 字符
   - 来源: 大辞泉 第二版


## Test 2: 获取所有音频信息

使用 `get_all_audio_info_from_mdx()` 获取词条的所有音频文件

In [11]:
# 测试获取所有音频
word = "政権"
mdx_path = DICTS_DIR / "DJS_N" / "DJS.mdx"

if mdx_path.exists():
    print(f"🔍 查询单词: {word}")
    print(f"📚 词典: 大辞泉 第二版\n")
    
    audio_infos = get_all_audio_info_from_mdx(mdx_path, word, "大辞泉")
    
    if audio_infos:
        print(f"✅ 找到 {len(audio_infos)} 个音频文件:\n")
        
        for i, info in enumerate(audio_infos, 1):
            print(f"   音频 {i}:")
            print(f"   - 路径: {info.audio_path}")
            print(f"   - 格式: {info.format}")
            print(f"   - MIME: {info.mime_type}")
            print(f"   - 大小: {len(info.audio_data)} bytes")
            print()
    else:
        print("ℹ️  未找到音频")
else:
    print("❌ 词典文件不存在")

🔍 查询单词: 政権
📚 词典: 大辞泉 第二版

✅ 找到 1 个音频文件:

   音频 1:
   - 路径: s00003472.aac
   - 格式: aac
   - MIME: audio/aac
   - 大小: 6529 bytes



## Test 3: 音调信息提取 (旧版 NHK)

从旧版 NHK 提取 tune-0/tune-1/tune-2 标记

In [12]:
# 旧版 NHK 路径 (根据实际情况调整)
nhk_old_dir = DICTS_DIR / "NHK_Old"

# 查找第一个 .mdx 文件
nhk_old_mdx = None
if nhk_old_dir.exists():
    mdx_files = list(nhk_old_dir.glob("*.mdx"))
    if mdx_files:
        nhk_old_mdx = mdx_files[0]

if nhk_old_mdx and nhk_old_mdx.exists():
    print(f"📚 旧版 NHK: {nhk_old_mdx.name}\n")
    
    # 测试多个单词
    test_words = ["精霊", "政権", "庇う"]
    
    for word in test_words:
        print(f"🔍 单词: {word}")
        
        reading, pitch_pos = extract_pitch_info_nhk_old(nhk_old_mdx, word)
        
        if reading:
            print(f"   ✅ 找到音调信息")
            print(f"   - 读音 (HTML): {reading}")
            print(f"   - 音调类型: {pitch_pos}")
            
            # 显示渲染效果
            display(HTML(f'<div style="font-size: 20px; padding: 10px; background: #f0f0f0; border-radius: 5px;">{reading}</div>'))
        else:
            print(f"   ℹ️  未找到音调信息")
        
        print()
else:
    print("❌ 未找到旧版 NHK 词典")
    print(f"   搜索目录: {nhk_old_dir}")

📚 旧版 NHK: NHKJPVDL.mdx

🔍 单词: 精霊
   ✅ 找到音调信息
   - 读音 (HTML): セ<span style="text-decoration: overline;">イ</span><span style="text-decoration: overline;">レ</span><span style="text-decoration: overline;">イ</span>
   - 音调类型: [0]



🔍 单词: 政権
   ✅ 找到音调信息
   - 读音 (HTML): セ<span style="text-decoration: overline;">イ</span><span style="text-decoration: overline;">ケ</span><span style="text-decoration: overline;">ン</span>
   - 音调类型: [0]



🔍 单词: 庇う
   ✅ 找到音调信息
   - 读音 (HTML): カ<span style="text-decoration: overline;">バ</span>ウ
   - 音调类型: [2]


## Test 4: AudioLookup 高层 API

使用 `AudioLookup` 类进行多词典联合查询

## Test 3.5: 获取所有音调信息 (多读音)

有些词有多个读音/音调,使用 `return_all=True` 获取所有音调信息

In [13]:
# 测试获取所有音调信息
if nhk_old_mdx and nhk_old_mdx.exists():
    print(f"📚 旧版 NHK: {nhk_old_mdx.name}\\n")
    
    # 测试多读音单词
    test_words = ["精霊", "政権", "庇う", "空模様"]
    
    for word in test_words:
        print(f"🔍 单词: {word}")
        
        # 获取所有读音的音调信息
        all_pitch_infos = extract_pitch_info_nhk_old(nhk_old_mdx, word, return_all=True)
        
        if all_pitch_infos:
            print(f"   ✅ 找到 {len(all_pitch_infos)} 个读音:\\n")
            
            for i, (reading, pitch_pos) in enumerate(all_pitch_infos, 1):
                # 移除 HTML 标签用于显示
                import re
                plain_reading = re.sub(r'<[^>]+>', '', reading)
                print(f"   读音 {i}: {plain_reading} {pitch_pos}")
                
                # 显示渲染效果
                display(HTML(f'<div style="font-size: 18px; padding: 8px; background: #f8f8f8; border-radius: 5px; margin: 5px 0;">{i}. {reading} <span style="color: #888; margin-left: 10px;">{pitch_pos}</span></div>'))
        else:
            print(f"   ℹ️  未找到音调信息")
        
        print()
else:
    print("❌ 未找到旧版 NHK 词典")

📚 旧版 NHK: NHKJPVDL.mdx\n
🔍 单词: 精霊
   ✅ 找到 3 个读音:\n
   读音 1: ショーリョー [2]


   读音 2: ショーリョー [0]


   读音 3: セイレイ [0]



🔍 单词: 政権
   ✅ 找到 1 个读音:\n
   读音 1: セイケン [0]



🔍 单词: 庇う
   ✅ 找到 1 个读音:\n
   读音 1: カバウ [2]



🔍 单词: 空模様
   ℹ️  未找到音调信息

   ℹ️  未找到音调信息



In [14]:
# 初始化 AudioLookup
audio_lookup = AudioLookup.from_dirs(
    nhk_old_dir=DICTS_DIR / "NHK_Old",
    djs_dir=DICTS_DIR / "DJS_N",
    dict_names={
        "DJS.mdx": "大辞泉 第二版",
    }
)

print("✅ AudioLookup 初始化完成")
print(f"   音频词典: {len(audio_lookup.audio_dicts)} 个")
for mdx_path, name in audio_lookup.audio_dicts:
    print(f"   - {name} ({'✅' if mdx_path.exists() else '❌'})")
print(f"   音调词典: {'✅ ' + audio_lookup.pitch_dict.name if audio_lookup.pitch_dict else '❌ 未配置'}")
print()

✅ AudioLookup 初始化完成
   音频词典: 2 个
   - NHK旧版 (✅)
   - 大辞泉 第二版 (✅)
   音调词典: ✅ NHKJPVDL.mdx



In [15]:
# 测试查询
test_words = ["政権", "精霊", "庇う"]

for word in test_words:
    print("=" * 60)
    print(f"🔍 查询单词: {word}")
    print("=" * 60)
    
    result = audio_lookup.lookup(word, verbose=True)
    
    # 显示结果
    if result['audio_base64']:
        print(f"\n🔊 音频信息:")
        print(f"   - 来源: {result['audio_source']}")
        print(f"   - 类型: {result['audio_mime']}")
        print(f"   - 大小: {len(result['audio_base64'])} 字符 (base64)")
    else:
        print(f"\n❌ 未找到音频")
    
    if result['reading']:
        print(f"\n📝 音调信息:")
        print(f"   - 来源: {result['pitch_source']}")
        print(f"   - 类型: {result['pitch_position']}")
        print(f"   - 读音:")
        display(HTML(f'<div style="font-size: 24px; padding: 10px; background: #f5f5f5; border-radius: 5px;">{result["reading"]}</div>'))
    else:
        print(f"\n❌ 未找到音调信息")
    
    print()

🔍 查询单词: 政権
✅ 音频: NHK旧版 (audio/ogg)
✅ 音调: NHK旧版 [0]

🔊 音频信息:
   - 来源: NHK旧版
   - 类型: audio/ogg
   - 大小: 6964 字符 (base64)

📝 音调信息:
   - 来源: NHK旧版
   - 类型: [0]
   - 读音:



🔍 查询单词: 精霊
✅ 音频: NHK旧版 (audio/ogg)
✅ 音调: NHK旧版 [0]

🔊 音频信息:
   - 来源: NHK旧版
   - 类型: audio/ogg
   - 大小: 6756 字符 (base64)

📝 音调信息:
   - 来源: NHK旧版
   - 类型: [0]
   - 读音:



🔍 查询单词: 庇う
✅ 音频: NHK旧版 (audio/ogg)
✅ 音调: NHK旧版 [2]

🔊 音频信息:
   - 来源: NHK旧版
   - 类型: audio/ogg
   - 大小: 4780 字符 (base64)

📝 音调信息:
   - 来源: NHK旧版
   - 类型: [2]
   - 读音:


In [16]:
# 测试查询 - 返回所有音调信息
print("\\n" + "=" * 60)
print("🔍 测试: 返回所有音调信息")
print("=" * 60 + "\\n")

test_word = "精霊"  # 选择一个可能有多读音的单词

# 查询并返回所有音调
result = audio_lookup.lookup(test_word, verbose=True, return_all_pitches=True)

if result.get('all_pitches'):
    print(f"\\n📝 找到 {len(result['all_pitches'])} 个读音:\\n")
    
    for i, (reading, pitch_pos) in enumerate(result['all_pitches'], 1):
        # 移除 HTML 标签用于显示
        import re
        plain_reading = re.sub(r'<[^>]+>', '', reading)
        print(f"   {i}. {plain_reading} {pitch_pos}")
        
        # 渲染效果
        display(HTML(f'''
        <div style="padding: 10px; background: #f5f9fc; border-left: 4px solid #2196F3; margin: 5px 0; border-radius: 4px;">
            <span style="font-size: 20px; font-weight: bold;">{i}.</span>
            <span style="font-size: 22px; margin-left: 10px;">{reading}</span>
            <span style="color: #2196F3; font-weight: bold; margin-left: 15px;">{pitch_pos}</span>
        </div>
        '''))
    
    print(f"\\n💡 提示: 默认使用第一个读音")
    print(f"   - reading: {result['reading'][:50]}...")
    print(f"   - pitchPosition: {result['pitch_position']}")
else:
    print("\\nℹ️  只找到一个读音或未找到音调信息")

\n============================================================
🔍 测试: 返回所有音调信息
============================================================\n
✅ 音频: NHK旧版 (audio/ogg)
✅ 音调: NHK旧版 找到 3 个读音
   1. ショーリョー [2]
   2. ショーリョー [0]
   3. セイレイ [0]
\n📝 找到 3 个读音:\n
   1. ショーリョー [2]


   2. ショーリョー [0]


   3. セイレイ [0]


\n💡 提示: 默认使用第一个读音
   - reading: <span style="text-decoration: overline;">シ</span><...
   - pitchPosition: [2]


## Test 5: 音频播放预览

在 Notebook 中直接播放提取的音频

In [17]:
# 选择一个单词测试
word = "政権"
result = audio_lookup.lookup(word)

print(f"🔍 单词: {word}\n")

if result['audio_base64']:
    print(f"🔊 音频来源: {result['audio_source']} ({result['audio_mime']})")
    
    # 构建 data URI
    audio_url = f"data:{result['audio_mime']};base64,{result['audio_base64']}"
    
    # 显示音频播放器
    audio_html = f'''
    <div style="padding: 20px; background: #f9f9f9; border-radius: 10px; border: 1px solid #ddd;">
        <h3 style="margin-top: 0;">🔊 {word}</h3>
        <audio controls style="width: 100%;">
            <source src="{audio_url}" type="{result['audio_mime']}">
            您的浏览器不支持音频播放
        </audio>
        <p style="margin-bottom: 0; color: #666; font-size: 14px;">
            来源: {result['audio_source']} | 格式: {result['audio_mime']}
        </p>
    </div>
    '''
    
    display(HTML(audio_html))
else:
    print("❌ 未找到音频")

print()

# 显示音调标记
if result['reading']:
    print(f"📝 音调标记: {result['pitch_position']}")
    display(HTML(f'''
    <div style="padding: 15px; background: #e8f4f8; border-radius: 10px; border: 1px solid #b3d9e6;">
        <div style="font-size: 28px; font-weight: bold; margin-bottom: 5px;">{result['reading']}</div>
        <div style="color: #555; font-size: 16px;">音调类型: {result['pitch_position']}</div>
    </div>
    '''))
else:
    print("❌ 未找到音调信息")

🔍 单词: 政権

🔊 音频来源: NHK旧版 (audio/ogg)



📝 音调标记: [0]


## Test 6: Anki 字段格式转换

将查询结果转换为 AnkiConnect 可用的格式

In [18]:
# 查询单词
word = "精霊"
result = audio_lookup.lookup(word)

# 转换为 Anki 格式
anki_fields = audio_lookup.format_for_anki(result)

print(f"📋 Anki 字段格式 (单词: {word}):\n")
print("=" * 60)

# 显示字段
print(f"\n1️⃣  reading 字段:")
if anki_fields['reading']:
    print(f"   HTML: {anki_fields['reading']}")
    print(f"   渲染效果:")
    display(HTML(f'<div style="font-size: 24px; padding: 10px; background: #f5f5f5;">{anki_fields["reading"]}</div>'))
else:
    print(f"   (空)")

print(f"\n2️⃣  pitchPosition 字段:")
print(f"   {anki_fields['pitchPosition'] or '(空)'}")

print(f"\n3️⃣  audio 字段:")
if anki_fields['audio']:
    print(f"   文件名: {anki_fields['audio']['filename']}")
    print(f"   Base64长度: {len(anki_fields['audio']['data'])} 字符")
    print(f"   目标字段: {anki_fields['audio']['fields']}")
else:
    print(f"   (空)")

print("\n" + "=" * 60)

# 返回结果对象供后续程序使用
print("\n💡 提示: anki_fields 对象可以传递给其他程序使用")
print(f"\n返回的对象类型: {type(anki_fields)}")
print(f"可用字段: {list(anki_fields.keys())}")

# 显示对象
anki_fields

📋 Anki 字段格式 (单词: 精霊):


1️⃣  reading 字段:
   HTML: セ<span style="text-decoration: overline;">イ</span><span style="text-decoration: overline;">レ</span><span style="text-decoration: overline;">イ</span>
   渲染效果:



2️⃣  pitchPosition 字段:
   [0]

3️⃣  audio 字段:
   文件名: audio_-905922204766552976.ogg
   Base64长度: 6756 字符
   目标字段: ['audio']


💡 提示: anki_fields 对象可以传递给其他程序使用

返回的对象类型: <class 'dict'>
可用字段: ['audio', 'reading', 'pitchPosition']


{'audio': {'data': 'T2dnUwACAAAAAAAAAABOJQAAAAAAAOe5KKwBUFNwZWV4ICAgc3BlZXgtMS4yYmV0YTIAAAAAAAABAAAAUAAAAAB9AAACAAAABAAAAAIAAAD/////gAIAAAEAAAABAAAAAAAAAAAAAAAAAAAAT2dnUwAAAAAAAAAAAABOJQAAAQAAANNUFQoBKiEAAABFbmNvZGVkIHdpdGggU3BlZXggc3BlZXgtMS4yYmV0YTIAAAAAAU9nZ1MAACN5AAAAAAAATiUAAAIAAABIhNQ1MR8fIxwjOzs3HHFxcXFxYWFdcXFxcXFxcXFxQ01dXV1dTU1NXV1dXV1dXU1NQ0M7Q0N0wCNOswAAznOc5zmjtgta1gta1gta1gta1pO2AAAHdMGjTrMAAM5znOc5o7YLWtYLWtYLWtYLWtaTtgAAB3TBkuYoGAANmzZs2KA7DTc02ghFJUqVaPimqZbrpJ0IitfDdMGM73MSod6fWV+qHpc8o7x7b75ssUnbNUlEP3TBkORzCi5mlVW4KqTDzsmoYjfDzRJ1hOiHj3t/TJwx0JXDdMGU4NPOwU6vfqtemMDpo23crzfa8zRGZtC574gorqS7LaeLWeXVJRwxrtnKWLjB2LaxWgYU3Uuc5z90wZTlSM0Q+VuQCOEWns5PLcgHIW2/X9Lo1hLqLHruEORvcGkDnQZtHbFa2bFbOQFa1rFC2bSd+xSjv3TBlODDztCyv5l8bpeCprkt8CV/k34W5WbtBhx5O6r1/zdZcqdRBF0csRbWQORFEJhWsNrWtD90wYzkcxWfMmU9x5HR+SMKtUKdLZIISdsYpzw/dMGdyYmeVauaEbq6uqo+4EMYNsK0S+gYW/IWpD6qKFWRD3/uK4Z6BH7wpppeg4R6joTHf////ba5Vb//ayyCtjopXEahYWscWKT1m6VdxVh0bmgie8daXFg1hySKda68boptWhUNV8YUEdkc3/3vir90wZ1FdqKn

---

## 🎯 智能读音匹配总结

**功能说明:**
- 当 NHK 词典返回多个读音时,使用 **fugashi** 获取单词的实际读音
- 自动匹配最接近的音调信息
- 如果 fugashi 不可用或无法获取读音,则默认返回**最后一个读音**(通常是最常用的)

**示例对比:**

| 单词 | 所有候选读音 | 默认选择 (旧逻辑) | 智能选择 (新逻辑) |
|------|-------------|------------------|------------------|
| 精霊 | ①ショーリョー ②ショーリョー ③セイレイ | ①ショーリョー (不常用) | ③セイレイ (常用) ✅ |
| 政権 | ①セイケン | ①セイケン | ①セイケン |
| 庇う | ①カバウ | ①カバウ | ①カバウ |

**使用方式:**

```python
from mdx_utils import AudioLookup

# 初始化
lookup = AudioLookup.from_dirs(
    nhk_old_dir=Path("NHK_Old"),
    djs_dir=Path("DJS"),
)

# 默认查询 - 自动使用 fugashi 匹配最合适的读音
result = lookup.lookup("精霊")
print(result['reading'])  # セイレイ (智能匹配)

# 查看所有候选读音
result = lookup.lookup("精霊", return_all_pitches=True)
for reading, pitch in result['all_pitches']:
    print(f"{reading} {pitch}")
```

**优点:**
1. ✅ 自动选择最常用的读音
2. ✅ 无需手动配置优先级
3. ✅ 支持 fallback (fugashi 不可用时使用最后一个)
4. ✅ 完全向后兼容

## 总结

### ✅ 完成的功能

1. **底层 API**
   - `extract_audio_from_mdx()` - 单词典音频提取
   - `get_all_audio_info_from_mdx()` - 获取所有音频信息
   - `extract_pitch_info_nhk_old()` - 音调信息提取
   - `get_word_reading_with_fugashi()` - Fugashi读音获取
   - `match_best_pitch()` - 智能读音匹配

2. **高层 API**
   - `AudioLookup` 类 - 多词典联合查询
   - `format_for_anki()` - Anki 字段格式转换

3. **技术特性**
   - ✅ 复用 `mdxscraper.core.audio` 模块
   - ✅ 高内聚低耦合设计
   - ✅ 支持多种音频格式 (AAC, MP3, WAV...)
   - ✅ 完整的音调标记提取
   - ✅ Base64 编码自动处理
   - ✅ **多读音智能匹配 (Fugashi集成)**

### 📚 词典优先级

| 优先级 | 词典 | 音频格式 | 音调标记 |
|--------|------|----------|----------|
| 1 | 新版 NHK | AAC (高质量) | 不完整 |
| 2 | 旧版 NHK | MP3/OGG | ✅ 完整 (tune-0/1/2) |
| 3 | 大辞泉 | MP3 | 仅下降位置 |

### 🎵 音调标记系统

**旧版 NHK HTML 标签:**
- `tune-0`: 低音
- `tune-1`: 高音 → 转换为上划线
- `tune-2`: 下降音 → 上划线 + 记录位置

**Anki 字段:**
- `reading`: `<span style="text-decoration: overline;">高音部分</span>`
- `pitchPosition`: `[0]`, `[1]`, `[2]` 等

### 🤖 智能读音匹配 (新增)

**问题:** NHK词典将不常用读音放在前面
**解决:** 使用 fugashi 获取实际读音并智能匹配

**示例: 精霊**
- 所有候选: ①ショーリョー [2] → ②ショーリョー [0] → ③セイレイ [0]
- Fugashi读音: セイレイ
- **智能选择: セイレイ [0]** ✅

**Fallback:** 如果 fugashi 不可用,返回最后一个读音(通常是最常用的)

**安装:**
```bash
pip install fugashi unidic-lite
```

### 📖 相关文档

- [MULTI_PITCH_SUPPORT.md](../mdx_utils/MULTI_PITCH_SUPPORT.md) - 多读音支持说明
- [FUGASHI_INTEGRATION.md](../mdx_utils/FUGASHI_INTEGRATION.md) - Fugashi集成详解
- [AUDIO_FEATURE_SUMMARY.md](../mdx_utils/AUDIO_FEATURE_SUMMARY.md) - 音频功能总结
- [README.md](../mdx_utils/README.md) - 完整使用文档

---

**完成日期**: 2025-10-19  
**最后更新**: 集成 Fugashi 智能读音匹配